In [7]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [8]:
import pandas as pd
import numpy as np

# --- SECTION 1: LOAD AND PREPARE DATA ---
file_path = 'Survey_input.xlsx'

# Load only the necessary sheets
df_survey = pd.read_excel(file_path, sheet_name='Survey Results') # df = dataframe (Python's take on a table)
df_design = pd.read_excel(file_path, sheet_name='Design matrix')

# Clean Design Matrix
# variable = pd.read_excel(file, sheet_name=string)
df_design['Task'] = df_design['Task'].ffill() # when a group of rows are labelled together with one merged cell, this command copies the last read value to the next empty one 
df_design = df_design.dropna(subset=['Concept']).copy() # removes empty concept rows

# --- SECTION 2: CALCULATE SUMMATIONS ---
task_columns = [col for col in df_survey.columns if 'Task' in col] # counts no. of columns with task in it # [item for item in list if condition]
choice_counts = {} #empty dictionary created which holds data holds data in pair sets (a "key" and a "value")

# Tally up the survey choices
for task_idx, col in enumerate(task_columns, start=1): # starts a loop that goes through each col,assigns a count (task_idx) starting at 1 # for counter, item in enumerate(list, start_number):
    counts = df_survey[col].value_counts().to_dict()  # counts how many times a certain option was chosen and tallies it to a dictionary format # variable = dataframe[column].value_counts().to_dict()
    for concept_choice, count in counts.items():
        concept_key = 'None' if concept_choice == 4 else str(int(float(concept_choice))) # Handle 4 as 'None', others as integer strings
        choice_counts[(task_idx, concept_key)] = count #saves finalcpunts to the master distionary, 'choice_counts'

def clean_concept(x): #defines a custom reusable rule
    if str(x).lower() == 'none' or pd.isna(x): # str() forces convertion to string, lower() converts to lower case; pd.insa(x) checks if the excel cell was empty
        return 'None'
    return str(int(float(x)))

# Map the survey counts to the design matrix #!
df_design['Summation'] = df_design.apply(
    lambda row: choice_counts.get((int(float(row['Task'])), clean_concept(row['Concept'])), 0), axis=1
)

# Dummy Code start

attribute_columns = ['Cut width', 'Engine disp', 'Price']

# 1. Create a working copy of your design matrix
df_calc = df_design.copy()

# 2. Filter out the 'None' rows for the transformation
active_mask = df_calc['Concept'] != 'None'

# 3. Rename the numbers into distinct text labels (e.g., 'Price_Level_1')
for col in attribute_columns:
    # Convert the entire column to 'object' type first so it accepts text strings
    df_calc[col] = df_calc[col].astype(object)
    
    # Clean the numbers and convert them to strings
    clean_levels = df_calc.loc[active_mask, col].astype(float).astype(int).astype(str)
    
    # Apply the new text labels back to the dataframe
    df_calc.loc[active_mask, col] = col + "_Level_" + clean_levels

# 4. Generate the Dummy Variables (Grid of 0s and 1s)
df_dummies = pd.get_dummies(df_calc.loc[active_mask, attribute_columns], dtype=int)

# 5. Re-attach to the main table and fill the 'None' rows with zeroes
df_final = pd.concat([df_calc[['Task', 'Concept', 'Summation']], df_dummies], axis=1).fillna(0)

# Create a list of the 9 new column headers
dummy_columns = df_dummies.columns.tolist()

# Dummy Code end

# --- SECTION 3: NULL MODEL CALCULATION ---
# For a null model, all attribute part-worths (betas) are explicitly 0.
# Therefore, the total utility (Total V) is exactly 0.0 for every option.
df_design['Total V'] = 0.0

# Calculate exponent, probabilities, and Log Likelihood
df_design['exp(Vi)'] = np.exp(df_design['Total V']) # This equals 1.0 for all rows

# P(i) calculations
task_exp_sums = df_design.groupby('Task')['exp(Vi)'].transform('sum')
df_design['P(i)'] = df_design['exp(Vi)'] / task_exp_sums

# Log Likelihood components
df_design['Log L components'] = df_design['Summation'] * np.log(df_design['P(i)'] + 1e-10)

null_log_likelihood = df_design['Log L components'].sum()
print(f"Null Log Likelihood: {null_log_likelihood}")

Null Log Likelihood: -801.98697050872


In [9]:
from pymoo.core.problem import ElementwiseProblem
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.optimize import minimize
import numpy as np
import pandas as pd

# --- SECTION 4: BUILD THE PROBLEM ENVIRONMENT ---
class MNL_Optimization(ElementwiseProblem):
    def __init__(self, df, attr_cols):
        # Extract data from the dummy-coded dataframe
        self.X_matrix = df[attr_cols].values
        self.tasks = df['Task'].values
        self.summations = df['Summation'].values
        
        # We now have 9 variables to solve for
        n_variables = len(attr_cols)
        
        # Set boundaries for the beta values
        super().__init__(n_var=n_variables, n_obj=1, xl=-10, xu=10)

    def _evaluate(self, x, out, *args, **kwargs):
        # 1. Calculate Utilities: V = X * Betas
        V = np.dot(self.X_matrix, x)
        exp_V = np.exp(V)

        # 2. Calculate Probabilities using a temporary table
        temp_df = pd.DataFrame({
            'Task': self.tasks, 
            'exp_V': exp_V, 
            'Summation': self.summations
        })
        
        task_sums = temp_df.groupby('Task')['exp_V'].transform('sum')
        P_i = temp_df['exp_V'] / task_sums

        # 3. Calculate Log-Likelihood
        LL_components = temp_df['Summation'] * np.log(P_i + 1e-10)
        log_likelihood = LL_components.sum()

        # 4. Maximize LL by minimizing negative LL
        out["F"] = -log_likelihood

# --- SECTION 5: RUN THE NSGA-II SOLVER ---
# Initialize the custom problem using df_final and your 9 dummy columns
problem = MNL_Optimization(df_final, dummy_columns)

# Configure the algorithm settings
algorithm = NSGA2(pop_size=100)

# Execute the search (increased to 100 generations for the extra variables)
results = minimize(
    problem,
    algorithm,
    ('n_gen', 100), 
    seed=1,
    verbose=True   
)

# --- SECTION 6: DISPLAY RESULTS ---
print("\n--- OPTIMIZATION COMPLETE ---")
print(f"Maximum Log Likelihood: {-results.F[0]:.4f}")
print("\n--- DISCRETE PART-WORTHS (BETAS) ---")

# Print the 9 separate beta values matched to their respective levels
for attr, beta in zip(dummy_columns, results.X):
    print(f"{attr}: {beta:.4f}")

n_gen  |  n_eval  | n_nds  |      eps      |   indicator  
     1 |      100 |      1 |             - |             -
     2 |      200 |      1 |  0.000000E+00 |             f
     3 |      300 |      1 |  4.969322E+02 |         ideal
     4 |      400 |      1 |  0.000000E+00 |             f
     5 |      500 |      1 |  4.999994E+01 |         ideal
     6 |      600 |      1 |  3.6491417154 |         ideal
     7 |      700 |      1 |  0.000000E+00 |             f
     8 |      800 |      1 |  2.749066E+01 |         ideal
     9 |      900 |      1 |  4.157503E+01 |         ideal
    10 |     1000 |      1 |  0.000000E+00 |             f
    11 |     1100 |      1 |  0.5665000550 |         ideal
    12 |     1200 |      1 |  5.3195481315 |         ideal
    13 |     1300 |      1 |  0.3642100524 |         ideal
    14 |     1400 |      1 |  8.8609738052 |         ideal
    15 |     1500 |      1 |  1.324573E+01 |         ideal
    16 |     1600 |      1 |  0.000000E+00 |            